In [1]:
# Import the dependencies.
# Import the datetime module from the datetime library.
from datetime import datetime
import pandas as pd
import numpy as np
import requests
from citipy import citipy
import json
import subprocess
import psycopg2

In [2]:
# Import the API key.
from config import client_id, client_secret, redirect_url, base_url, oauth_url

url = "https://api.procore.com/oauth/token?grant_type=client_credentials&client_id=" + client_id + "&client_secret=" + client_secret

In [3]:
response = requests.post(url).json()
token = response.get('access_token')

In [4]:
get_company = "https://api.procore.com/rest/v1.0/companies"

data = requests.get(get_company, headers={"Authorization": "Bearer " + token}).json()

In [5]:
print(data)

[{'id': 7930, 'is_active': True, 'name': 'Oden Hughes Taylor Construction, LLC.'}]


In [6]:
get_Vendors = "https://api.procore.com/rest/v1.0/vendors?company_id=7930"

Vendors = requests.get(get_Vendors, headers={"Authorization": "Bearer " + token}).json()

In [7]:
vendors_df = pd.json_normalize(Vendors)

vendors_df

,id,abbreviated_name,address,attachments,authorized_bidder,bidding_distribution,business_phone,business_register,children_count,city,...,primary_contact.created_at,primary_contact.email_address,primary_contact.first_name,primary_contact.last_name,primary_contact.updated_at,business_register.id,business_register.identifier,business_register.type,business_register.verification_status,business_register.verified_at
0,5058709,,"901 S. Mopac Expressway\r\nBOP 3, Ste. 220","[{'id': 1634843716, 'filename': 'Procore - New...",True,"[{'id': 7687379, 'company_name': 'Oden Hughes ...",(512) 813-7139,NaN,0,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5058730,,"901 S. Mopac Expressway, Bldg 3, Ste 220",[],False,[],(512) 813-7111,NaN,0,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5058736,None,,[],False,[],,NaN,0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5115759,None,"901 S. Mopac Expressway, BOP. III, Ste. 220",[],False,[],(512) 813-7111,NaN,0,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5232492,,,[],False,[],,NaN,0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,23429653,None,3709 Promontory Point Dr. Ste b-248,[],True,"[{'id': 1467865, 'company_name': 'SET Central ...",(512) 382-0690,NaN,0,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,23430636,None,1121 Enclave Way,[],True,"[{'id': 7403103, 'company_name': 'Enclave Envi...",(512) 884-3463,NaN,0,Hutto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,23447661,None,"9811 Katy Freeway, Suite 925",[],True,[],(713) 491-2860,NaN,0,Houston,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999,23463279,,605 W. Howard Lane Ste 101,"[{'id': 3149467156, 'filename': 'W9.pdf', 'nam...",True,"[{'id': 6038946, 'company_name': 'Kompan, Inc....",(800) 426-9788,NaN,0,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
vendors_df.columns

Index(['id', 'abbreviated_name', 'address', 'attachments', 'authorized_bidder',
       'bidding_distribution', 'business_phone', 'business_register',
       'children_count', 'city', 'company', 'contact_count', 'country_code',
       'created_at', 'email_address', 'fax_number', 'is_active', 'labor_union',
       'legal_name', 'license_number', 'logo', 'mobile_phone', 'name',
       'non_union_prevailing_wage', 'notes', 'origin_code', 'origin_data',
       'origin_id', 'parent', 'prequalified', 'primary_contact', 'project_ids',
       'standard_cost_codes', 'state_code', 'synced_to_erp', 'trade_name',
       'trades', 'union_member', 'updated_at', 'vendor_group', 'website',
       'zip', 'bidding.affirmative_action',
       'bidding.african_american_business', 'bidding.asian_american_business',
       'bidding.certified_business_enterprise',
       'bidding.disadvantaged_business', 'bidding.eight_a_business',
       'bidding.hispanic_business',
       'bidding.historically_underutilized

In [9]:
select_vendors_df = vendors_df[["address", "id", "name", "trade_name"]]

select_vendors_df

,address,id,name,trade_name
0,"901 S. Mopac Expressway\r\nBOP 3, Ste. 220",5058709,"Oden Hughes Taylor Construction, LLC.",
1,"901 S. Mopac Expressway, Bldg 3, Ste 220",5058730,Subcontracting TEST Company,
2,,5058736,Architect TEST Company,
3,"901 S. Mopac Expressway, BOP. III, Ste. 220",5115759,"Oden Hughes, LLC",
4,,5232492,Silver Ventures,
...,...,...,...,...
1996,3709 Promontory Point Dr. Ste b-248,23429653,"SET Central Texas, LLC",
1997,1121 Enclave Way,23430636,Enclave Environmental,
1998,"9811 Katy Freeway, Suite 925",23447661,"Slate Leander, LP",
1999,605 W. Howard Lane Ste 101,23463279,"Kompan, Inc.",


In [10]:
select_vendors_df = select_vendors_df.fillna(' ')

In [11]:
select_vendors_df.drop(index=0)

,address,id,name,trade_name
1,"901 S. Mopac Expressway, Bldg 3, Ste 220",5058730,Subcontracting TEST Company,
2,,5058736,Architect TEST Company,
3,"901 S. Mopac Expressway, BOP. III, Ste. 220",5115759,"Oden Hughes, LLC",
4,,5232492,Silver Ventures,
5,"3636 Executive Center Drive, Suite 209\r\n\r\n",5330135,"CSF Civil Group, LLC",
...,...,...,...,...
1996,3709 Promontory Point Dr. Ste b-248,23429653,"SET Central Texas, LLC",
1997,1121 Enclave Way,23430636,Enclave Environmental,
1998,"9811 Katy Freeway, Suite 925",23447661,"Slate Leander, LP",
1999,605 W. Howard Lane Ste 101,23463279,"Kompan, Inc.",


In [12]:
conn = psycopg2.connect(
   database="Master_DB", user='OdenAdmin', password= "aJDbULW45C93zCykCxAp", host='oht-rds.cbunnafql9ym.us-east-2.rds.amazonaws.com', port= '5432'
)

#Setting auto commit false
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Define SQL execution command
insert_into_table = ("""INSERT INTO "OHT_companies" ( "CompanyID", "CompanyAddress", "CompanyName", "CompanyType") VALUES(%s,%s,%s,%s);""")

# Iterate through DF and execute commands to add data to table
for i, row in select_vendors_df.iterrows():
   row_to_insert = ( row['id'], row['address'], row['name'], row['trade_name'])
   cursor.execute(insert_into_table, row_to_insert)

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()